In [1]:
import numpy as np
import os
from pathlib import Path

import adcraft.gymnasium_kw_env as kw_sim
from adcraft.experiment_utils.experiment_quantiles import (
    make_experiment_quantiles, load_experiment_quantiles
)

In [2]:
def construct_observations_manually(env, agent_bids: dict, days_passed: int = 0, cumulative_profit: float = 0.0):
    """Prints outcomes for simplified manually bidding for a day. 
    
    It doesn't take into account the sub-time steps so its results are somewhat simplified when budget plays a role. 
    When budget is high enough not to affect results, they should be about the same. 
    This is used to generate the numbers in Appendix F: Example of Bidding Outcomes.
    The exact numbers may not be identical when run due to randomness, but the structure of the outcomes will be computed identically.
    """
    observation, _ =  env.reset()
    for k in range(env.num_keywords):
        print(f"Manually generating keyword {k} observations.")
        volume = env.keywords[k].volume_sampler()
        print(f"Total auction volume: {volume}")

        bids = env.keywords[k].bid_distribution(1, volume)
        print(f"Critical bids to win each auction:\n {bids}")

        agent_bid = agent_bids[k]
        num_impressions = np.sum(bids <= agent_bid)
        print(f"(Oberserved) number of impressions: {num_impressions}")
        impressions = bids[bids <= agent_bid]
        print(impressions)
        observation["impressions"][k] = num_impressions

        clicks_occured = env.keywords[k].rng.random((num_impressions)) <= env.keywords[k].buyside_ctr
        num_clicks = np.sum(clicks_occured)
        costs = impressions[clicks_occured]
        total_cost = np.sum(costs)
        print(f"(Observed) Number of clicked impressions: {num_clicks}")
        observation["buyside_clicks"][k] = num_clicks
        print(f"Cost of each individual click:\n {costs}")
        print(f"(Observed) Total cost of clicks: {total_cost}")
        observation["cost"][k] = total_cost

        conversions_occured = env.keywords[0].rng.random((num_clicks)) <= env.keywords[k].sellside_paid_ctr
        num_conversions = np.sum(conversions_occured)
        revenues = env.keywords[k].reward_distribution_sampler(num_conversions)
        total_revenue = np.sum(revenues)
        print(f"(Observed) Total conversions: {num_conversions}")
        observation["sellside_conversions"][k] = num_clicks
        print(f"Revenue for each individual conversion:\n {revenues}")
        print(f"(Observed) Total revenue of conversions: {total_revenue}")
        observation["revenue"][k] = total_revenue
        print(f"(not in observation vector, but calculable) Profit on keyword {k}: {total_revenue - total_cost}\n")
        cumulative_profit += total_revenue - total_cost
    observation["cumulative_profit"] = cumulative_profit
    observation["days_passed"] = days_passed + 1
    return observation

In [3]:
# Run this code block to actually produce results like in Appendix F. (They won't be identical)
env = kw_sim.bidding_sim_creator(
    env_config=dict(
        keyword_config={
            "outer_directory": Path.cwd().as_posix()+"",
            "mean_volume": 16,
            "conversion_rate": 0.5,
            "make_quant_func": make_experiment_quantiles,
            "load_quant_func": load_experiment_quantiles
        },
        num_keywords = 2,
        max_days = 10,
        updater_params= [["vol", 0.03], ["ctr", 0.03], ["cvr", 0.03]],
        updater_mask = None
    )
)
obs, info = env.reset(seed=0)
print(info["keyword_params"])

observations = construct_observations_manually(env, [0.75, 0.75], 0, 0.0)
print("\n".join([f"{k}: "+str(observations[k]) for k in ["impressions", "buyside_clicks", "cost", "sellside_conversions", "revenue", "days_passed", "cumulative_profit"]]))        

print("\nnext day\n\n")
observations = construct_observations_manually(env, [0.59, 0.75], 1, observations["cumulative_profit"])
print("\n".join([f"{k}: "+str(observations[k]) for k in ["impressions", "buyside_clicks", "cost", "sellside_conversions", "revenue", "days_passed", "cumulative_profit"]]))      

kw0 params:
 volume: (16, 1),   imp_intercept: 0.6459721981904619,   imp_slope: 9.492169932038324,   bctr: 0.7526828432972257,   sctr: 0.5,   mean revenue: 1.229655446429944,   std revenue: 0.3184237989333203
kw1 params:
 volume: (16, 8),   imp_intercept: 0.7565469048855986,   imp_slope: 4.711917056603076,   bctr: 0.10219080013611848,   sctr: 0.5,   mean revenue: 0.5459178688435826,   std revenue: 0.08867800662594895
Manually generating keyword 0 observations.
Total auction volume: 17
Critical bids to win each auction:
 [[0.67 0.6  0.62 0.81 0.56 0.68 0.46 0.76 0.74 0.57 0.79 0.42 0.6  0.52
  0.63 0.74 0.56]]
(Oberserved) number of impressions: 14
[0.67 0.6  0.62 0.56 0.68 0.46 0.74 0.57 0.42 0.6  0.52 0.63 0.74 0.56]
(Observed) Number of clicked impressions: 12
Cost of each individual click:
 [0.67 0.6  0.62 0.56 0.68 0.46 0.74 0.57 0.6  0.52 0.63 0.56]
(Observed) Total cost of clicks: 7.209999999999999
(Observed) Total conversions: 5
Revenue for each individual conversion:
 [0.88 0.8